In [20]:
import matplotlib.pyplot as plt
import h5py
import os
import scipy.signal as signal
from scipy import sparse
from scipy.sparse.linalg import spsolve
import glob
import numpy as np
import shutil
from decimal import Decimal
from PIL import Image
import mne
import pandas as pd
##code from https://github.com/Naereen/Lempel-Ziv_Complexity/blob/master/src/lempel_ziv_complexity.py

In [4]:
eeg= mne.io.read_raw_eeglab('a.set')#, preload=True)
print ((eeg.annotations[0]))
print (np.shape(eeg.get_data()))
print (eeg.get_data()[0])

OrderedDict([('onset', 0.013671875), ('duration', 39.9921875), ('description', 'boundary'), ('orig_time', None)])
(64, 31702)
[-1.79053887e-02  4.22420508e-03  6.43819727e-03 ...  2.43238029e-05
  3.26363564e-05  2.53985863e-05]


<ipython-input-4-05c8f0a5a105>:1: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  eeg= mne.io.read_raw_eeglab('a.set')#, preload=True)
<ipython-input-4-05c8f0a5a105>:1: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg= mne.io.read_raw_eeglab('a.set')#, preload=True)


In [13]:
i= 17
eeg_arr= eeg.get_data()[i]- np.median(eeg.get_data()[i])
print (np.median(eeg.get_data()[i]))

2.9735003411769863e-07


In [14]:
for i in range(0, len(eeg_arr)):
    if eeg_arr[i]<= 0:
        eeg_arr[i]= 0
        
    else :
        eeg_arr[i]= 1


In [15]:
def lempel_ziv_complexity(binary_sequence):
    """ Manual implementation of the Lempel-Ziv complexity.
    It is defined as the number of different substrings encountered as the stream is viewed from begining to the end.
    As an example:
    >>> s = '1001111011000010'
    >>> lempel_ziv_complexity(s)  # 1 / 0 / 01 / 1110 / 1100 / 0010
    6
    Marking in the different substrings the sequence complexity :math:`\mathrm{Lempel-Ziv}(s) = 6`: :math:`s = 1 / 0 / 01 / 1110 / 1100 / 0010`.
    - See the page https://en.wikipedia.org/wiki/Lempel-Ziv_complexity for more details.
    Other examples:
    >>> lempel_ziv_complexity('1010101010101010')  # 1 / 0 / 10
    3
    >>> lempel_ziv_complexity('1001111011000010000010')  # 1 / 0 / 01 / 1110 / 1100 / 0010 / 000 / 010
    7
    >>> lempel_ziv_complexity('100111101100001000001010')  # 1 / 0 / 01 / 1110 / 1100 / 0010 / 000 / 010 / 10
    8
    - Note: it is faster to give the sequence as a string of characters, like `'10001001'`, instead of a list or a numpy array.
    - Note: see this notebook for more details, comparison, benchmarks and experiments: https://Nbviewer.Jupyter.org/github/Naereen/Lempel-Ziv_Complexity/Short_study_of_the_Lempel-Ziv_complexity.ipynb
    - Note: there is also a Cython-powered version, for speedup, see :download:`lempel_ziv_complexity_cython.pyx`.
    """
    u, v, w = 0, 1, 1
    v_max = 1
    length = len(binary_sequence)
    complexity = 1
    while True:
        if binary_sequence[u + v - 1] == binary_sequence[w + v - 1]:
            v += 1
            if w + v >= length:
                complexity += 1
                break
        else:
            if v > v_max:
                v_max = v
            u += 1
            if u == w:
                complexity += 1
                w += v_max
                if w > length:
                    break
                else:
                    u = 0
                    v = 1
                    v_max = 1
            else:
                v = 1
    return complexity

In [18]:
complexity= lempel_ziv_complexity((eeg_arr))
print (complexity)

792


In [17]:
print (len(str(eeg_arr)))

23


In [19]:
n_complexity= complexity*np.log2(len(eeg.get_data()[0]))/len(eeg.get_data()[0])
print (n_complexity)

0.37354774781209826
